## Manual *yt* selections with dask arrays

This notebook is an initial field test of returning dask arrays when accessing fields in a *yt* dataset.

It uses the https://github.com/chrishavlin/yt/tree/dask_init_particle branch with a few small modifications. First, in `BaseIOHandler._read_particle_selection`, it does not call compute on the dask arrays in the field dictionary, so that delayed arrays are returned. Second, the following code in `data_selection_objects.YTSelectionContainer`:

```python
        for f, v in read_particles.items():
            self.field_data[f] = self.ds.arr(v, units=finfos[f].units)
            self.field_data[f].convert_to_units(finfos[f].output_units)            
```    

is replaced with 

```python
        from unyt import dask_array

        for f, v in read_particles.items():
            da_f = dask_array.unyt_from_dask(v, units=finfos[f].units, registry=self.ds.unit_registry)
            self.field_data[f] = da_f.to(finfos[f].output_units)
```      

this will result in returing `unyt_dask` arrays! 

In [1]:
import yt

In [2]:
ds = yt.load_sample("snapshot_033")
ad = ds.all_data()

yt : [INFO     ] 2021-06-03 16:31:38,204 Files located at /home/chris/hdd/data/yt_data/yt_sample_sets/snapshot_033.tar.gz.untar/snapshot_033/snap_033.
yt : [INFO     ] 2021-06-03 16:31:38,205 Default to loading snap_033.0.hdf5 for snapshot_033 dataset
yt : [INFO     ] 2021-06-03 16:31:38,302 Parameters: current_time              = 4.343952725460923e+17 s
yt : [INFO     ] 2021-06-03 16:31:38,303 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2021-06-03 16:31:38,303 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2021-06-03 16:31:38,304 Parameters: domain_right_edge         = [25. 25. 25.]
yt : [INFO     ] 2021-06-03 16:31:38,304 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2021-06-03 16:31:38,305 Parameters: current_redshift          = -4.811891664902035e-05
yt : [INFO     ] 2021-06-03 16:31:38,305 Parameters: omega_lambda              = 0.762
yt : [INFO     ] 2021-06-03 16:31:38,305 Parameters: omega_matter              = 0.238
yt : [

In [3]:
den = ad[("PartType4","Density")]  # will use hmsl = 0 
den

unyt_dask_array<mul, shape=(155926,), dtype=float64, chunksize=(20772,), chunktype=numpy.ndarray, units=code_mass/code_length**3>

cool! we have our `unyt_dask` array! Can do dask and unyt things:

In [4]:
den = den.to('kg/m**3')
den

unyt_dask_array<mul, shape=(155926,), dtype=float64, chunksize=(20772,), chunktype=numpy.ndarray, units=kg/m**3>

In [5]:
den.mean()

unyt_dask_array<mean_agg-aggregate, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray, units=kg/m**3>

In [6]:
den.mean().compute()

unyt_quantity(4.28121508e-21, 'kg/m**3')

Ok, that's kinda neat. 

### slicing instead of selection objects?? 

Now, this is happening on the `all_data()` selection object. If we wanted to do a sphere selection, we could of course do:

In [7]:
sp = ds.sphere(ds.domain_center,ds.quan(5,'code_length'))


In [8]:
%%time
den_sp = sp[("PartType4","Density")]

CPU times: user 3.07 s, sys: 30.6 ms, total: 3.1 s
Wall time: 3.08 s


In [9]:
den_sp

unyt_dask_array<mul, shape=(29205,), dtype=float64, chunksize=(4293,), chunktype=numpy.ndarray, units=code_mass/code_length**3>

and what yt does behind the scenes is apply the selection object to each chunk of the dask array, so that we only return the values within the array. Note that the initial instantiation of `den_sp` actually takes a bit of time -- that's because creating the dask array requires knowing the length of each chunk that will be concatenated into our total dask array. So even though we get a delayed array, there is an initial embedded compute to get the expected lengths.

**Ok, that's all well and good**, but since our dask array doesn't actually hold the array in memory until we call compute, we can actually do our selections with array-slicing syntax, and dask will go and slice by each chunk, kind of similar to how the yt native selection objects work. 

Let's pull out the coordinates from all the data:

In [10]:
xyz = ad[("PartType4","Coordinates")]
xyz

unyt_dask_array<mul, shape=(155926, 3), dtype=float64, chunksize=(20772, 3), chunktype=numpy.ndarray, units=code_length>

and manually calculate a distance from the center. As it turns out, it seems that there's a bug in the new unyt dask arrays, where the array becomes a normal dask array when slicing. So we'll do these operations in a unyt-less way:

In [11]:
import numpy as np 

C = ds.domain_center.value
R = float(ds.quan(5,'code_length').value)

In [12]:
dist = np.sqrt( (xyz[:,0] - C[0])**2 + (xyz[:,1] - C[1])**2 + (xyz[:,2]- C[2])**2 )
dist

dask.array<sqrt, shape=(155926,), dtype=float64, chunksize=(20772,), chunktype=numpy.ndarray>

and now we get a new `unyt_dask` array for density (so we get back to the initial units) and mask out our sphere:

In [13]:
den = ad[("PartType4","Density")]  
den_sp_manual = den[dist <= R]
den_sp_manual

dask.array<getitem, shape=(nan,), dtype=float64, chunksize=(nan,), chunktype=numpy.ndarray>

Let's pull our density into memory for our manually sliced sphere:

In [14]:
%%time
den_in_mem = den_sp_manual.compute()

CPU times: user 45.4 ms, sys: 8.81 ms, total: 54.3 ms
Wall time: 48.8 ms


and now for our yt-natively selected sphere:

In [15]:
%%time
den_sp_selector = den_sp.compute()

CPU times: user 21.8 ms, sys: 273 µs, total: 22.1 ms
Wall time: 16.9 ms


do our arrays match?

In [16]:
den_in_mem

array([ 7156342.   , 15433073.   ,  2540943.   , ...,    47703.96 ,
          37973.906,    36136.465], dtype=float32)

In [17]:
den_sp_selector.value

array([ 7156342.   , 15433073.   ,  2540943.   , ...,    47703.96 ,
          37973.906,    36136.465], dtype=float32)

In [18]:
den_in_mem.shape

(29205,)

In [19]:
den_sp_selector.shape

(29205,)

In [20]:
np.all(den_in_mem == den_sp_selector.value)

True

yes! we get the same selection!

One thing that the yt native selection object does that the manual dask array method does not do is limit the chunks that are checked. The dataset indexing records the spatial regions covered by each chunk, so that if the large scale chunk does not intersect the selection object, it doesnt bother checking that chunk and saves some computation there. The dask-slicing approach will check each chunk, so it does some extra work there but it should be possible to add some indexing logic to avoid checking chunks. 

A further complication is that some particle types use a "smoothing length" that may be a bit harder to adapt to a slicing syntax.


All that said, the dask slicing method is faster in this case because the pre-allocation is much faster for `all_data` (because it just reads an attribute from the hdf file). To emphasize this, here are all the above operations collected together for the standard way:

In [22]:
%%timeit
sp = ds.sphere(ds.domain_center,ds.quan(5,'code_length'))
den_sp = sp[("PartType4","Density")]
den_sp_selector = den_sp.compute()

3.18 s ± 101 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


and the dask-slicing method:

In [23]:
%%timeit
ad = ds.all_data()
xyz = ad[("PartType4","Coordinates")]

C = ds.domain_center.value
R = float(ds.quan(5,'code_length').value)
dist = np.sqrt( (xyz[:,0] - C[0])**2 + (xyz[:,1] - C[1])**2 + (xyz[:,2]- C[2])**2 )

den = ad[("PartType4","Density")]  
den_sp_manual = den[dist <= R]
den_in_mem = den_sp_manual.compute()


48.6 ms ± 6.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


that's an impressive speedup... but one caveat is that the slowness for the native approach could be coming from inefficient pickling protocol. The selection object must be pickled and passed to the selection routines for dask to do it's thing, so I suspect that improving how that works could speed up the native approach. 

A final note: by "native" I actually mean "daskified-native", as the branch I'm working on has dask functionality within the particle reader. 